In [1]:
%pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [5]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [9]:
from abc import abstractstaticmethod
##########################################
# --- Agente 1: Buscador de Artigos --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa acadêmico. A sua tarefa é usar as ferramentas de busca do google (google-search)
        para procurar e listar artigos, dissertações de mestrado, e teses de doutorados que sejam relevantes sobre o tópico abaixo.
        Foque em no mínimo 3 e no máximo 6 resultados. Podem ser resultados de todo lugar do mundo, mas escritos em português. indicar para cada um dos resultados se trata-se de
        um artigo, uma dissertação de mestrado, uma tese de douturado, ou outros. Se possível, indicar o link de onde foi retirado o resultado das buscas.
        """,
        description="Agente que busca artigos no Google",
        # Corrige o nome do parâmetro para 'tools' e passa o objeto Tool
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    artigos = call_agent(buscador, entrada_do_agente_buscador)
    return artigos

In [10]:
################################################
# --- Agente 2: Resumo dos Artigos --- #
################################################
def agente_resumo(topico, artigos):
    resumos = Agent(
        name="agente_resumo",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Resumo #################################################
        instruction="""Você é um especialista em resumos acadêmicos. Com base nos artigos encontrados, você deve levantar os pontos mais relevantes
        de cada artigo, dissertação de mestrado ou tese de doutorado, e redigir um resumo sussinto sobre cada um deles. No início de cada resumo,
        inserir o título do artigo, o autor, a data e a universidade ou faculdade. Para cada um dos resumos, iniciar com uma parte descritiva,
        seguir com alguns tópicos importantes e elaborar uma conclusão. Você também pode usar o (google_search) para enconrtar
        mais informações sobre os temas e aprofundar (mas indicar quando as informações vieram de fora do artigo). O tom deve ser jovem e direto.
        """,

        description="Agente que resume artigos",
        tools=[google_search]
    )

    entrada_do_agente_resumo = f"Tópico:{topico}\nArtigos_encontrados: {artigos}"
    # Executa o agente
    resumos = call_agent(resumos, entrada_do_agente_resumo)
    return resumos

In [11]:
######################################
# --- Agente 3: Redator de Post --- #
######################################
def agente_redator(topico, resumos):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts para o Linkedin.
            Você escreve posts e textos acadêmicos. Utilize os resumos dos artigos, identifique os pontos mais relevantes,
            e escreva um rascunho de post para o linkedin com o tema indicado. O post não deve ser muito extenso.
            O post deve ser engajador, informativo, com linguagem num tom mais profissional. Explicite qual é a principal descoberta ou insight
            da pesquisa. Incentive debates. Incluir 2 a 4 hashtags no final. Importante inserir no final do post a referencia bibliografica.
            """,
        description="Agente redator de posts de resumos de artigos no Linkedin"
    )

    entrada_do_agente_redator = f"Tópico: {topico}\nPlano_de_post: {resumos}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [12]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para o linkedin.
            Por ser um publico de jobs, use um tom de escrita adequado.
            Revise o rascunho de post sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para Linkedin."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [14]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para o Linkedin 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o topico!")
else:
  print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

  artigos = agente_buscador(topico, data_de_hoje)
  print("n--- Resultado do Agente 1 (Buscador de Artigos) ---\n")
  display(to_markdown(artigos))
  print("----------------------------------------------------------")

  resumos = agente_resumo(topico, artigos)
  print("n--- Resultado do Agente 2 (Resumos dos Artigos) ---\n")
  display(to_markdown(resumos))
  print("----------------------------------------------------------")

  rascunho = agente_redator(topico, resumos)
  print("n--- Resultado do Agente 3 (Rasculho do post do Linkedin) ---\n")
  display(to_markdown(rascunho))
  print("----------------------------------------------------------")

  texto_revisado = agente_revisor(topico, rascunho)
  print("n--- Resultado do Agente 4 (Revisor de Conteúdo) ---\n")
  display(to_markdown(texto_revisado))




🚀 Iniciando o Sistema de Criação de Posts para o Linkedin 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: 
Você esqueceu de digitar o topico!
